# Lab 7: Principal Component Analysis (PCA)

In this lab session we will implement the Principal Component Analysis (PCA) to reduce the dimensionality of the Iris flower dataset.

We will first code this ourselves using NumPy and will then validate our solution comparing our results with Scipy's PCA implementation.

Finally, we will compare the PCA-reduced samples to the reduced samples we obtained by manually selecting two features in lab 5.

As usual, let's import the libraries before we start by running the cell below.

In [1]:
from __future__ import print_function # to avoid issues between Python 2 and 3 printing

import numpy as np
import matplotlib.pyplot as plt

from pprint import pprint
from sklearn.decomposition import PCA

# show matplotlib figures inline
%matplotlib inline

In [2]:
# By default we set figures to be 12"x8" on a 110 dots per inch (DPI) screen 
# (adjust DPI if you have a high res screen!)
plt.rc('figure', figsize=(12, 8), dpi=110)
plt.rc('font', size=12)

## Load the data

In this lab we will use the Iris dataset again. Let's run the cell below to load the data.

In [3]:
# load the iris train and test sets

def load_iris_data(train_path='iris_train.csv', test_path='iris_test.csv'):
    train_set = np.loadtxt(train_path, delimiter=',')
    test_set = np.loadtxt(test_path, delimiter=',')

    # separate labels from features
    train_labels = train_set[:, 4].astype(np.int)
    train_set = train_set[:, 0:4]
    test_labels = test_set[:, 4].astype(np.int)
    test_set = test_set[:, 0:4]
    
    return train_labels, train_set, test_labels, test_set

train_labels, train_set, test_labels, test_set = load_iris_data()

## PCA steps

To implement PCA we will need to implement the following steps __on the training set__ which is represented as a $n \times d$ matrix :

1. Calculate the covariance matrix 
2. Calculate the eigenvectors and the corresponding eigenvalues. 
3. Sort the eigenvectors by decreasing eigenvalues and choose the first $k$ eigenvectors using such order. This is to create a $d \times k$ matrix $W$ that will be used to project the data into a new lower dimensional space.
4. Use the $W$ matrix to transform the samples onto the new space.

## 1. Calculate the covariance matrix

You can use NumPy's function [`np.cov`](https://docs.scipy.org/doc/numpy/reference/generated/numpy.cov.html) for this. Pay attention to the `rowvar` argument and make sure your covariance matrix is 4x4.

In [9]:

cov = np.cov(train_set, rowvar = False)




## 2. Calculate eigenvectors and eigenvalues

You can use NumPy's function [`np.linalg.eig`](https://docs.scipy.org/doc/numpy/reference/generated/numpy.linalg.eig.html?highlight=eig#numpy.linalg.eig) for this

In [5]:
ei = np.linalg.eig(cov)
eiVal = ei[0]
eiVec = ei[1]
print(eiVec)

[[ 0.37956727 -0.64272351 -0.57294026  0.33848876]
 [-0.0710188  -0.73690217  0.59691517 -0.30923097]
 [ 0.85220314  0.18159765  0.07431063 -0.48502581]
 [ 0.35303659  0.10442207  0.55669531  0.74468217]]


## 3. Sort the eigenvectors by decreasing eigenvalues

The eigenvalues provide a measure of how much information each eigvenvector carries. By sorting the eigenvectors in descending order according to the respective eigenvalues, we thus sort the various components in decreasing order of importance.

There is no guarantee `np.linalg.eig` returns the eigenvectors in a sorted order, so we need to to sort them before creating the $W$ matrix.

Choose the first $k$ eigenvectors using such order to create the $d \times k$ matrix $W$. Print the sorted eigenvectors and the corresponding eigenvalues to check your code. 

We will use $k=2$, so you should obtain a matrix of shape $4 \times 2$.

__Hint__. There are different ways to achieve this. As a suggestion, our solution used NumPy's functions [`np.flip`](https://docs.scipy.org/doc/numpy/reference/generated/numpy.flip.html) and [`np.argsort`](https://docs.scipy.org/doc/numpy/reference/generated/numpy.argsort.html).

In [6]:
eiValD = -np.sort(-eiVal)

order = []

eiVecIm = np.zeros(eiVec.shape)
#print(eiVecIm)
j = 0
for i in range(np.size(eiVal)):
    
    if eiVal[j] == eiValD[i]:
        order.append(i)
        j+=1

    
        
        
print(order)
z = 0
for x in range(np.size(eiVal)):
    #print(x, z)
    
    eiVecIm[z] = eiVec[x]
    
    z += 1
    
#print(eiVecIm)


w = np.array([eiVecIm[0], eiVecIm[1]])
#w1 = w.transpose()
print(eiVec)
#print(w1)

    
    
    
    

[0, 1, 2, 3]
[[ 0.37956727 -0.64272351 -0.57294026  0.33848876]
 [-0.0710188  -0.73690217  0.59691517 -0.30923097]
 [ 0.85220314  0.18159765  0.07431063 -0.48502581]
 [ 0.35303659  0.10442207  0.55669531  0.74468217]]


NameError: name 'w1' is not defined

## 4. Use the $W$ matrix to transform the samples onto the new space.

Transform __both__ the training and test set. Remember that you can use the `dot` function defined for NumPy arrays to perform matrix multiplication on two arrays.

In [ ]:
#print(train_set)
#w1 = w.transpose
#print(w)
Trans = np.dot(train_set, eiVec)
#Trans2 = np.dot(Trans, train_set)

#print(Trans)
plt.scatter(Trans[:,0], -Trans[:,1])

## 5. Use SciPy's PCA

We will now validate our solution comparing it with SciPy's implementation. SciPy provides a class called [`PCA`](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html), which we already imported in this labsheet.

Steps to follow to use Scipy's PCA:

1. Create the PCA object, specifying the number of components (in our case 2). 
2. Using the created object, [`fit`](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html#sklearn.decomposition.PCA.fit) the PCA model on the training set.
3. [`Transform`](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html#sklearn.decomposition.PCA.transform) both the training and test sets using the fitted model.

Once you've completed the steps above, produce two scatter plots comparing the training set reduced with your PCA to that reduced with Scipy's PCA. You should obtain a plot similar to the one below (we are providing the colour codes we used for the plot). 

Most importantly, __your scatter plots should be identical__ (axis scale aside) to prove that your implementation is correct.


![](pca.png)

### Note

Scipy's transformed data will be flipped horizontally. This is due to the fact that the eigenvectors can have either a positive or a negative sign, depending on how they are calculated. 

To obtain the above plot and thus have two identical scatter plots, we flipped Scipy's reduced data along the y axis (tip: you simple need to multiple the second column by `-1` to do that). Note that this does not tamper the data, since what matters is how the samples are located in relationship one with the other. By flipping the samples we are preserving the distribution of the data.

Note that the y axis scale of the two plots is also different. This is because Scipy scales the data to have unit length 1. Again, what matters is how the data is distributed, so we can safely tell that our implementation is correct!

To obtain the same scaling as our plots, use `ax.set_aspect('equal')`.

In [ ]:
CLASS_1_C = r'#3366ff'
CLASS_2_C = r'#cc3300'
CLASS_3_C = r'#ffc34d'

pca = PCA(n_components=2)
pca.fit(train_set)
#print(pca.singular_values_)
y1 = pca.transform(train_set) 
#print(y1)


plt.scatter(y1[:,0], -y1[:,1])


## 6. Compare with your selected features

### Scatter plots

Let's finally consider the two features you selected in lab 5. We want to compare the manually reduced dataset to the PCA-reduced one (use the dataset reduced with your own implementation).

To do this, we will first compare the two scatter plots. You should obtain something like this:

![](pca_comparison.png)

In [ ]:
# write your code here



### Nearest-Centroid accuracy

By looking at the above plots, it seems like our manually reduced dataset separates the data in a better way. 

Let's prove this by running the Nearest-Centroid classifier you implemented in lab 5, and let's calculate the accuracy as we did in lab 6.

In [ ]:
# write your code here  k


### Discussion

Using features (3, 4) and our PCA-reduced dataset, we obtained the following results

```
Accuracy with manually selected features: 0.96
Accuracy with PCA: 0.88
```

In fact, as suspected before, the PCA results are slightly worse. This should not come to a big surprise.

PCA excels when dealing with high dimensionality data (e.g. 1024 or more dimensions). In our case, our simple Iris dataset contains 4 features only, and we already observed that, due to the simplicity of the dataset, by carefully selecting 2 features we can obtain nearly perfect results (i.e. 96% of test samples are correctly classified).

However, this is an ideal scenario. In real problems you will deal with far more complex and higher dimensional datasets, in which cases PCA will be helpful.